In [3]:
'''
Main Tracking Software

Michael Opheim, 1/16/2022
Adapted from Sergio Canu's motion tracking software

This program tracks and quantifies object movement within videos files.

Instructions:
1. Download 'tracker' software from https://pysource.com/2021/01/28/object-tracking-with-opencv-and-python/
2. In cv2.VideoCapture, type in the title of your mp4 file.
3. Find the dimensions of your video by unquoting the 'print(height, width)' command.
4. Use these dimensions to create your specific region of interest, or 'roi' variable (the exact region you would like the program to monitor for movement).
5. Run the program to track and collect the relative (x,y) coordinates of your desired moving object(s).

Tips:
1. Press the 'q' key to end the program at any time.
2. The number assignments corresponding to the variables 'varThreshold' and 'area' can be adjusted to reduce the "noise" your video.
3. Y-coordinates are, by default, backwards. To fix this, change the 'directions.append' command to 'detections.append([x, height - y, w, h])'.
    - Note: this will remove the tracking boxes from your video, so be sure to utilize this change after you have made sure that your object tracking is working as intended.
4. Issues and errors can be reported to michaeldopheim@gmail.com, where tech support will be provided.

'''

import cv2
from tracker import *

#creates tracker object
tracker = EuclideanDistTracker()

#reads frames from the video
capture = cv2.VideoCapture("Video_Title.mp4")

#extracts moving objects
object_detector = cv2.createBackgroundSubtractorMOG2(history = 100, varThreshold = 30) #threshold can be changed to remove false positives

#loops through video frames
while True:
    ret, frame = capture.read()
    height, width, _ = frame.shape #finds dimensions of video
    '''print(height, width)''' #use this to find height and width of the video
    
    if ret:
        roi = frame[605: 1080, 20: 1500] #extracts region of interest [height, width]
    
    #object detection
        mask = object_detector.apply(roi) #applies object detection to the region of interest ("roi" could also be replaced with the variable "frame", which scans the entire video)
        _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY) #removes the gray from the mask to stop detecting object shadows
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #finds contours in the mask
        detections = [] #where (x, y, w, h) coordinates are collected
        for cnt in contours:
            area = cv2.contourArea(cnt) #calculates object areas and removes small elements
            if area > 500: #if area is greater than 500 pixels...
                '''cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)''' #highlights contours as green ("roi" could also be replaced with the variable "frame")
                x, y, w, h = cv2.boundingRect(cnt)
                detections.append([x, y, w, h]) #adds coordinates to detections variable
        '''print(detections)''' #simply prints the detection coordinates
    
    #object tracking
        boxes_ids = tracker.update(detections) #gives a novel ID to newly detected objects
        '''print(boxes_ids)''' #simply prints coordinates with their object IDs
        for box_id in boxes_ids:
            x, y, w, h, id = box_id
            cv2.putText(roi, str(id), (x, y - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2) #places box ID on box in video frame
            cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3) #highlights contours with a green rectangle
    
        '''cv2.imshow("roi", roi)''' #shows the frames in the region of interest - use this to ensure that the region's dimensions are accurate
        cv2.imshow("Frame", frame) #shows the video's frames in real time
        '''cv2.imshow("Mask", mask)''' #shows the video in its mask format
        key = cv2.waitKey(30) #wait time in milliseconds before each frame is presented
        if key == ord('q'): #press the 'q' key to close the video window
            break
    else:
        break

capture.release()
cv2.destroyAllWindows()
for i in range (1,5):
    cv2.waitKey(1)